# DVS Hardpoint Analysis

This notebook computes several performance metrics for team **DVS** across their recorded Hardpoint maps.

In [1]:

import pandas as pd
import re

# Load dataset
df = pd.read_excel('DVS-HP.xlsx')

# Create numeric score column for DVS based on left side of 'Score'
def extract_dvs_score(row):
    left, right = map(int, row['Score'].split('-'))
    return left if row['Team1'] == 'DVS' else right

df['DVS_Score'] = df.apply(extract_dvs_score, axis=1)

# Apply differencing to the DVS_Score column, except for the first row
df['PointsScored'] = (
    df
    .groupby(['Map','Team2'])['DVS_Score']
    .diff()
    .fillna(df['DVS_Score'])                        # first row of each hill → 0 points
    .astype(int)
)

df.head()


,Map,Team1,Team2,Hill,RotateFirst,RotationWin,HoldDuration,BreakTeam,BreakSuccess,BreakDuration,ContestTime,ScrapTeam,ScrapTime,Score,DVS_Score,PointsScored
0,Slums,DVS,WL,P1,DVS,No,10,WL,No,3,2,Split,0,39-9,39,39
1,Slums,DVS,WL,P2,DVS,Yes,37,WL,No,4,2,DVS,10,85-13,85,46
2,Slums,DVS,WL,P3,WL,No,10,DVS,Yes,15,2,DVS,10,120-33,120,35
3,Slums,DVS,WL,P4,WL,No,2,DVS,Yes,57,2,DVS,8,175-36,175,55
4,Slums,DVS,WL,P1,DVS,Yes,30,WL,Yes,25,2,WL,10,198-61,198,23


## 1. Points scored per hill on each map

In [2]:
# Compute the average DVS points per hill per map
avg_points_per_hill = (
    df
    .groupby(['Map', 'Hill'])['PointsScored']
    .mean()                  # average instead of sum
    .round(2)                # round to 2 decimal places
    .reset_index()
    .pivot(index='Map', columns='Hill', values='PointsScored')
)

# Flatten the column index for easier manipulation
avg_points_per_hill.columns.name = None
avg_points_per_hill = avg_points_per_hill.reset_index()

avg_points_per_hill.to_csv('avg_points_per_hill.csv', index=False)


## 2. Rotation counts and wins

In [3]:

# Flag where DVS rotates first
df['DVS_RotatesFirst'] = df['RotateFirst'] == 'DVS'
# Flag rotation wins for DVS
df['DVS_RotationWin'] = (df['DVS_RotatesFirst']) & (df['RotationWin'] == 'Yes')

rot_summary_map = df.groupby('Map').agg(
    RotatedFirst=('DVS_RotatesFirst','sum'),
    RotationWins=('DVS_RotationWin','sum')
)
rot_summary_map



,RotatedFirst,RotationWins
Map,,
Combine,6,2
Hacienda,7,6
Slums,9,5


In [16]:

import numpy as np

# 1) Compute totals & percentages per Map/Hill
rot_stats = (
    df
    .groupby(['Map','Hill'])
    .agg(
        TotalRounds=('DVS_RotatesFirst','count'),
        RotatedFirst=('DVS_RotatesFirst','sum'),
        RotationWins=('DVS_RotationWin','sum'),
    )
)

# 2) Calculate percentages
rot_stats['RotatePct'] = rot_stats['RotatedFirst'] / rot_stats['TotalRounds'] * 100
rot_stats['WinPct']    = np.where(
    rot_stats['RotatedFirst'] > 0,
    rot_stats['RotationWins'] / rot_stats['RotatedFirst'] * 100,
    0
)

# 3) Pivot into two clean tables
rotate_pct_table = (
    rot_stats['RotatePct']
    .unstack(fill_value=0)
    .round(1)
)
rotation_win_pct_table = (
    rot_stats['WinPct']
    .unstack(fill_value=0)
    .round(1)
)

# 4) (Optional) Rename columns so they’re self-describing
rotate_pct_table.columns = [f"{hill}_RotatePct" for hill in rotate_pct_table.columns]
rotation_win_pct_table.columns = [f"{hill}_WinPct" for hill in rotation_win_pct_table.columns]

# 5) Inspect
rotate_pct_table.to_csv('rotate_pct_table.csv')
rotation_win_pct_table.to_csv('rotation_win_pct_table.csv')

In [5]:
# Concatenate the two tables side by side
rotation_stats = pd.concat(
    [rotate_pct_table, rotation_win_pct_table],
    axis=1
).reset_index()

# Save the rotation stats to a CSV file
rotation_stats.to_csv('rotation_stats.csv', index=False)

## 3. Break attempts and successes

In [6]:

# Break attempts where DVS is BreakTeam
df['DVS_BreakAttempt'] = df['BreakTeam'] == 'DVS'
df['DVS_BreakSuccess'] = (df['DVS_BreakAttempt']) & (df['BreakSuccess'] == 'Yes')

break_summary_map = df.groupby('Map').agg(
    BreakAttempts=('DVS_BreakAttempt','sum'),
    BreakSuccesses=('DVS_BreakSuccess','sum')
)
break_summary_map


,BreakAttempts,BreakSuccesses
Map,,
Combine,5,0
Hacienda,5,1
Slums,8,4


In [17]:
# 1) Compute totals & percentages per Map/Hill
break_stats = (
    df
    .groupby(['Map','Hill'])
    .agg(
        TotalRounds=('DVS_BreakAttempt','count'),
        BreakAttempts=('DVS_BreakAttempt','sum'),
        BreakSuccesses=('DVS_BreakSuccess','sum'),
    )
)

# 2) Calculate percentages
break_stats['BreakAttemptPct'] = break_stats['BreakAttempts'] / break_stats['TotalRounds'] * 100
break_stats['BreakSuccessPct'] = np.where(
    break_stats['BreakAttempts'] > 0,
    break_stats['BreakSuccesses'] / break_stats['BreakAttempts'] * 100,
    0
)

# 3) Pivot into two clean tables
break_attempt_pct = (
    break_stats['BreakAttemptPct']
    .unstack(fill_value=0)
    .round(1)
)
break_success_pct = (
    break_stats['BreakSuccessPct']
    .unstack(fill_value=0)
    .round(1)
)

# 4) Rename columns for clarity
break_attempt_pct.columns = [f"{hill}_BreakAttemptPct" for hill in break_attempt_pct.columns]
break_success_pct.columns = [f"{hill}_BreakSuccessPct" for hill in break_success_pct.columns]

# 5) Inspect results
break_attempt_pct.to_csv('break_attempt_pct.csv')
break_success_pct.to_csv('break_success_pct.csv')


In [9]:
# Concatenate the two tables side by side
break_stats_table = pd.concat(
    [break_attempt_pct, break_success_pct],
    axis=1
).reset_index()

break_stats_table.to_csv('break_stats.csv', index=False)

## 4. Average hold durations

In [10]:
# First hold avg when DVS rotates first
avg_first_hold_map = (
    df[df['DVS_RotatesFirst']]
    .groupby('Map')['HoldDuration']
    .mean()
)
avg_first_hold_map


Map
Combine     10.000000
Hacienda    23.000000
Slums       16.555556
Name: HoldDuration, dtype: float64

In [11]:
avg_first_hold_hill = (
    df[df['DVS_RotatesFirst']]
    .groupby(['Map','Hill'])['HoldDuration']
    .mean()
    .round(1)
    .reset_index()
    .pivot(index='Map', columns='Hill', values='HoldDuration')
    .rename_axis(None, axis=1)
    .reset_index()
    .fillna(0.0)  # Fill NaN with 0.0 for clarity
)
avg_first_hold_hill.to_csv('avg_first_hold_hill.csv', index=False)


In [12]:

# Average break hold duration where DVS is BreakTeam
avg_break_hold_map = (
    df[df['DVS_BreakAttempt']]
    .groupby('Map')['BreakDuration']
    .mean()
)
avg_break_hold_map


Map
Combine      2.800
Hacienda     7.200
Slums       17.625
Name: BreakDuration, dtype: float64

In [13]:
avg_break_hold_hill = (
    df[df['DVS_BreakAttempt']]
    .groupby(['Map','Hill'])['BreakDuration']
    .mean()
    .round(1)
    .reset_index()
    .pivot(index='Map', columns='Hill', values='BreakDuration')
    .rename_axis(None, axis=1)
    .reset_index()
    .fillna(0.0)  # Fill NaN with 0.0 for clarity
)

avg_break_hold_hill.to_csv('avg_break_hold_hill.csv', index=False)

## 5. Scrap time collection

In [14]:

df['DVS_MajorityScrap'] = df['ScrapTeam'] == 'DVS'
df['ScrapSplit'] = df['ScrapTeam'] == 'Split'

scrap_summary = pd.DataFrame({
    'TotalHills': [len(df)],
    'HillsWithMajorityScrap': [df['DVS_MajorityScrap'].sum()],
    'HillsWithSplitScrap': [df['ScrapSplit'].sum()]
})

scrap_summary

,TotalHills,HillsWithMajorityScrap,HillsWithSplitScrap
0,40,17,4


In [18]:
# 2) Scrap time per hill per map (total seconds DVS collects)
scrap_per_hill = (
    df[df['DVS_MajorityScrap']]                        # only rows where DVS got majority scrap
      .groupby(['Map','Hill'])['ScrapTime']
      .sum()                                           # sum scrap seconds
      .round(1)                                        # optional rounding
      .unstack(fill_value=0)                          # pivot hills as columns
      .reset_index()                                  # make Map a column again
)

scrap_per_hill.to_csv('scrap_per_hill.csv', index=False)